# Top 10 suburbs in Perth Analysis

As part of the IBM Datascience course that I am taking, I've decided to do an analysis of the 
top suburbs in Perth, Australia based on the list from this site: https://www.homely.com.au/search/suburbs-in-perth-greater-western-australia, and find a business to start in one of the suburbs. 

Perth is one of the top cities that asians are emigrating to. 
I hope this simple analysis will provide an insight to anyone looking to leverage on a first mover advantage to fill a business need in the suburb.

## Data Preprocessing

I've created a spreadsheet of the 10 suburbs and their postal codes, which was then downloaded as a CSV file
and we will work on the dataset from there

In [8]:
# Import libraries and dataset
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
sns.set()

# Reading the content of the CSV file into a Pandas Dataframe
df = pd.read_csv('/Users/yslim/Downloads/suburbs - List.csv')
df

,Postcode,Suburb
0,6009,Dalkeith
1,6156,Willagee
2,6149,Bull Creek
3,6150,Murdoch
4,6153,Mount Pleasant
5,6025,Cragie
6,6171,Baldivis
7,6014,Floreat
8,6027,Edgewater
9,6107,Wilson


## Getting the Latitude and Longitude
To explore each suburb further later, we will need their lats and longs

In [9]:
import geocoder # import geocoder

pos = df['Postcode'].tolist()

lats = []
longs = []

for p in pos:
    
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Perth, Australia'.format(p))
        lat_lng_coords = g.latlng
        
    lats.append(lat_lng_coords[0])
    longs.append(lat_lng_coords[1])
    
df['Latitude'] = lats
df['Longitude'] = longs

In [22]:
# Showing the current dataframe
df

,Postcode,Suburb,Latitude,Longitude
0,6009,Dalkeith,-31.983247,115.807295
1,6156,Willagee,-32.037436,115.804335
2,6149,Bull Creek,-32.064419,115.870580
3,6150,Murdoch,-32.062759,115.834665
4,6153,Mount Pleasant,-32.024410,115.839269
5,6025,Cragie,-31.801190,115.756343
6,6171,Baldivis,-32.331321,115.833210
7,6014,Floreat,-31.936620,115.800860
8,6027,Edgewater,-31.752955,115.757133
9,6107,Wilson,-32.018825,115.952937


## Visualising on a Map
Let's plot our locations on a Folium map. 

In [10]:
# First, get the coordinates of Perth
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

address = 'Perth'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Perth are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Perth are -31.9527121, 115.8604796.


In [19]:
# Let's see our suburbs on a map
import folium # map rendering library

# create map of New York using latitude and longitude values
map_perth = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, suburb in zip(df['Latitude'], df['Longitude'], df['Suburb']):
    label = '{}'.format(suburb)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_perth)  
    
map_perth

## Further Analysis with Foursquare API 

#### Setting up the login credentials first

In [21]:
CLIENT_ID = 'KW2GPGLKGNL3Y2ZPWKUVUJRSPCKDHM1FV4ULZFPZ3GAKHXKF' # your Foursquare ID
CLIENT_SECRET = 'FZ1M0BOJ1PJO4YZVVRLQMRKKJULAN3POJIFR3MODL2SLKHDB' # your Foursquare Secret
VERSION = '20190924'
LIMIT = 30

#### We are going to explore the venues around the first suburb: Floreat

In [58]:
latitude = df.loc[7, 'Latitude']
longitude = df.loc[7, 'Longitude']

#### Getting the top 30 venues (locations) that are in Floreat within a radius of 1000 meters.¶

In [54]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# type your answer here

all_venues = getNearbyVenues(names=df['Suburb'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


# print the dataframe
all_venues.tail(10)

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
252,Wilson,-32.018825,115.952937,Salsa's Fresh Mex Grill,-32.018075,115.937182,Fast Food Restaurant
253,Wilson,-32.018825,115.952937,Aldi,-32.021310,115.937290,Supermarket
254,Wilson,-32.018825,115.952937,Geláre,-32.019930,115.938484,Ice Cream Shop
255,Wilson,-32.018825,115.952937,Subway,-32.027007,115.946998,Sandwich Place
256,Wilson,-32.018825,115.952937,Spotlight,-32.026189,115.951305,Arts & Crafts Store
257,Wilson,-32.018825,115.952937,Coles,-32.020959,115.939465,Supermarket
258,Wilson,-32.018825,115.952937,Nando's,-32.011154,115.937644,Fast Food Restaurant
259,Wilson,-32.018825,115.952937,Miss Maud,-32.017683,115.938699,Café
260,Wilson,-32.018825,115.952937,Myer,-32.019135,115.938072,Department Store
261,Wilson,-32.018825,115.952937,Beckenham IGA,-32.013973,115.965390,Grocery Store


In [55]:
all_venues.shape

(262, 7)

#### Getting the top 30 venues (locations) that are in all suburbs within a radius of 2000 meters.¶

In [24]:
# Constructing the GET request 
LIMIT = 30
radius = 1000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

# Getting the results back from the GET request
import json
results = requests.get(url).json()


# Just a function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
    
# Cleaning the results and presenting it in a Pandas dataframe
venues = results['response']['groups'][0]['items']    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

# Showing the top 5 results here
nearby_venues.head()

,name,categories,lat,lng
0,Ciao Italia,Café,-31.984245,115.814772
1,The Tenth State,Café,-31.977779,115.814084
2,Chez Pierre,French Restaurant,-31.979790,115.797910
3,Windsor Cinema,Indie Movie Theater,-31.979502,115.802151
4,Nando's,Portuguese Restaurant,-31.980006,115.797568


#### Grouping all suburbs and showing the number of venues each suburb offers

In [43]:
#all_venues[all_venues.Suburb == "Floreat"]
all_venues.groupby('Suburb').count()

,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Suburb,,,,,,
Baldivis,5,5,5,5,5,5
Bull Creek,30,30,30,30,30,30
Cragie,28,28,28,28,28,28
Dalkeith,30,30,30,30,30,30
Edgewater,30,30,30,30,30,30
Floreat,30,30,30,30,30,30
Mount Pleasant,30,30,30,30,30,30
Murdoch,19,19,19,19,19,19
Willagee,30,30,30,30,30,30


## Creating a Dataframe with top 8 most common venues in each Suburb

#### Firstly, create a table with categories of suburbs

In [46]:
# one hot encoding
suburb_onehot = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
suburb_onehot['Suburb'] = all_venues['Suburb'] 

# move neighborhood column to the first column
fixed_columns = [suburb_onehot.columns[-1]] + list(suburb_onehot.columns[:-1])
suburb_onehot = suburb_onehot[fixed_columns]

suburb_onehot.head()

,Suburb,African Restaurant,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Bar,Basketball Stadium,Beer Store,...,Stadium,Supermarket,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Vegetarian / Vegan Restaurant,Video Store,Water Park,Wine Bar
0,Dalkeith,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Dalkeith,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Dalkeith,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Dalkeith,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Dalkeith,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by suburb and by taking the mean of the frequency of occurrence of each category¶

In [47]:
suburb_grouped = suburb_onehot.groupby('Suburb').mean().reset_index()
suburb_grouped.head()

,Suburb,African Restaurant,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Bar,Basketball Stadium,Beer Store,...,Stadium,Supermarket,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Vegetarian / Vegan Restaurant,Video Store,Water Park,Wine Bar
0,Baldivis,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000
1,Bull Creek,0.0,0.000000,0.033333,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.100000,0.0,0.033333,0.0,0.0,0.0,0.033333,0.0,0.000000
2,Cragie,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.107143,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000
3,Dalkeith,0.0,0.000000,0.000000,0.033333,0.033333,0.033333,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.033333
4,Edgewater,0.0,0.033333,0.033333,0.000000,0.000000,0.033333,0.0,0.0,0.0,...,0.0,0.033333,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000


In [56]:
# This function sorts the venues in a descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


# Creating the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 8
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Suburb']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Suburb'] = suburb_grouped['Suburb']

for ind in np.arange(suburb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(suburb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,Baldivis,Café,Sandwich Place,Fast Food Restaurant,Supermarket,Wine Bar,Dive Bar,Donut Shop,Electronics Store
1,Bull Creek,Supermarket,Liquor Store,Japanese Restaurant,Fast Food Restaurant,Chinese Restaurant,Shopping Mall,Grocery Store,Big Box Store
2,Cragie,Fast Food Restaurant,Supermarket,Coffee Shop,Department Store,Park,Electronics Store,Restaurant,Big Box Store
3,Dalkeith,Café,Pub,Grocery Store,Italian Restaurant,Pizza Place,Coffee Shop,Wine Bar,Hospital
4,Edgewater,Coffee Shop,Electronics Store,Department Store,College Cafeteria,Burger Joint,Café,Furniture / Home Store,Juice Bar
5,Floreat,Coffee Shop,Café,Supermarket,Gym,Sandwich Place,Burger Joint,Wine Bar,Pub
6,Mount Pleasant,Café,Park,Japanese Restaurant,Ice Cream Shop,Burger Joint,Hawaiian Restaurant,Water Park,Italian Restaurant
7,Murdoch,Shopping Mall,Chinese Restaurant,Malay Restaurant,Department Store,Café,Pharmacy,Middle Eastern Restaurant,Park
8,Willagee,Japanese Restaurant,Café,Indian Restaurant,Grocery Store,Fast Food Restaurant,Supermarket,Farmers Market,Comfort Food Restaurant
9,Wilson,Supermarket,Grocery Store,Gym / Fitness Center,Fast Food Restaurant,Department Store,Bubble Tea Shop,Electronics Store,Juice Bar


## Exploring a Suburb: Floreat 

Floreat is a 24 minutes drive away from the main airport in Perth, and is just a 10 minutes drive away
from the city centre. 

It seems to be lacking a Pub (8th Most Common Venue). 

Let's explore more of this suburb

### Search for a specific venue category

#### Checking for all pubs available in this suburb, within a 5000m radius

In [61]:
# Setting up the lats and longs of Floreat
latitude = df.loc[7, 'Latitude']
longitude = df.loc[7, 'Longitude']

# Defining the search parameters
search_query = 'Pub'
radius = 5000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Durty Nelly's Irish Pub,Pub,937 Murray St,AU,Perth,Australia,Shafto Ln,5267,"[937 Murray St (Shafto Ln), Perth WA 6000, Aus...","[{u'lat': -31.951805921, u'lng': 115.853676174...",-31.951806,115.853676,NaN,6000,WA,4b383f8cf964a520274d25e3
1,Mayfair Lane Pub & Dining Room,Gastropub,72 Outram St,AU,West Perth,Australia,Mayfair St,3909,"[72 Outram St (Mayfair St), West Perth WA 6005...","[{u'lat': -31.9473535118, u'lng': 115.84026893...",-31.947354,115.840269,NaN,6005,WA,4d8f46dbc1791456b5e7e213
2,Packer Pub,Lounge,6955 Stonewick dr.,AU,Newburgh,Australia,NaN,2541,"[6955 Stonewick dr., Newburgh IN 47630, Austra...","[{u'lat': -31.9563132548, u'lng': 115.81448078...",-31.956313,115.814481,NaN,47630,IN,5068a142fe70906d363ede3b
3,Rafa's Pub,Beer Garden,Spotted Wy,AU,NaN,Australia,NaN,3109,"[Spotted Wy, WA 6038, Australia]","[{u'lat': -31.917134224, u'lng': 115.824445471...",-31.917134,115.824445,Mt Hawthorn,6038,Western Australia,520d00f511d275108fb1283b
4,VIP Publishers,Design Studio,NaN,AU,NaN,Australia,NaN,811,[Australia],"[{u'lat': -31.939377, u'lng': 115.792904, u'la...",-31.939377,115.792904,NaN,NaN,NaN,4c57c1b130d82d7f589bda62
5,Subiaco Hotel,Pub,Hay St.,AU,Subiaco,Australia,at Rokeby Rd.,2502,"[Hay St. (at Rokeby Rd.), Subiaco WA 6008, Aus...","[{u'lat': -31.9470502135, u'lng': 115.82433075...",-31.947050,115.824331,NaN,6008,WA,4d2ea935a6df6dcb6a94db7a
6,Chubb Pub,Bar,NaN,AU,NaN,Australia,NaN,4904,[Australia],"[{u'lat': -31.9262251166, u'lng': 115.85131284...",-31.926225,115.851313,NaN,NaN,NaN,4eead0ae61af22f158bc84c4
7,An Sibin Irish Pub,Pub,147 James Street,AU,Northbridge,Australia,Lake Street,5443,"[147 James Street (Lake Street), Northbridge W...","[{u'lat': -31.94818, u'lng': 115.85685, u'labe...",-31.948180,115.856850,NaN,6000,WA,5493ed9b498e2fe76beff93b
8,Clancy's Fish Bar,Pub,195 Challenger Parade,AU,City Beach,Australia,City Beach,4284,"[195 Challenger Parade (City Beach), City Beac...","[{u'lat': -31.936850352, u'lng': 115.755504033...",-31.936850,115.755504,NaN,6015,WA,4d451bc91ed56dcbaac9b754
9,Professional Public Relations,Office,588 Hay Street,AU,Perth,Australia,Price Street,1777,"[588 Hay Street (Price Street), Perth WA, Aust...","[{u'lat': -31.94675, u'lng': 115.81541, u'labe...",-31.946750,115.815410,NaN,NaN,WA,4d7d6a4cf260a0938a574cba


#### Let's check our categories to see if the search has included any irrelevant results

In [63]:
dataframe_filtered.categories.unique

<bound method Series.unique of 0                         Pub
1                   Gastropub
2                      Lounge
3                 Beer Garden
4               Design Studio
5                         Pub
6                         Bar
7                         Pub
8                         Pub
9                      Office
10                    Library
11    States & Municipalities
12    States & Municipalities
13                     Office
14                Pizza Place
15                    Library
16                     Lawyer
17                        Bar
18         Mexican Restaurant
Name: categories, dtype: object>

#### As you can see, the search term includes any categories with 'pub' in it.
#### Let's filter out the dataframe to get any results that are actual drinking spots

In [80]:
list = ['Pub', 'Bar','Gastropub','Beer Garden']
pubsdf =  dataframe_filtered[dataframe_filtered.categories.isin(list)]

pubsdf.head(12)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Durty Nelly's Irish Pub,Pub,937 Murray St,AU,Perth,Australia,Shafto Ln,5267,"[937 Murray St (Shafto Ln), Perth WA 6000, Australia]","[{u'lat': -31.951805921, u'lng': 115.853676174, u'label': u'display'}]",-31.951806,115.853676,NaN,6000,WA,4b383f8cf964a520274d25e3
1,Mayfair Lane Pub & Dining Room,Gastropub,72 Outram St,AU,West Perth,Australia,Mayfair St,3909,"[72 Outram St (Mayfair St), West Perth WA 6005, Australia]","[{u'lat': -31.9473535118, u'lng': 115.84026893, u'label': u'display'}]",-31.947354,115.840269,NaN,6005,WA,4d8f46dbc1791456b5e7e213
3,Rafa's Pub,Beer Garden,Spotted Wy,AU,NaN,Australia,NaN,3109,"[Spotted Wy, WA 6038, Australia]","[{u'lat': -31.917134224, u'lng': 115.824445471, u'label': u'display'}]",-31.917134,115.824445,Mt Hawthorn,6038,Western Australia,520d00f511d275108fb1283b
5,Subiaco Hotel,Pub,Hay St.,AU,Subiaco,Australia,at Rokeby Rd.,2502,"[Hay St. (at Rokeby Rd.), Subiaco WA 6008, Australia]","[{u'lat': -31.9470502135, u'lng': 115.824330757, u'label': u'display'}]",-31.947050,115.824331,NaN,6008,WA,4d2ea935a6df6dcb6a94db7a
6,Chubb Pub,Bar,NaN,AU,NaN,Australia,NaN,4904,[Australia],"[{u'lat': -31.9262251166, u'lng': 115.851312842, u'label': u'display'}]",-31.926225,115.851313,NaN,NaN,NaN,4eead0ae61af22f158bc84c4
7,An Sibin Irish Pub,Pub,147 James Street,AU,Northbridge,Australia,Lake Street,5443,"[147 James Street (Lake Street), Northbridge WA 6000, Australia]","[{u'lat': -31.94818, u'lng': 115.85685, u'label': u'display'}]",-31.948180,115.856850,NaN,6000,WA,5493ed9b498e2fe76beff93b
8,Clancy's Fish Bar,Pub,195 Challenger Parade,AU,City Beach,Australia,City Beach,4284,"[195 Challenger Parade (City Beach), City Beach WA 6015, Australia]","[{u'lat': -31.936850352, u'lng': 115.755504033, u'label': u'display'}]",-31.936850,115.755504,NaN,6015,WA,4d451bc91ed56dcbaac9b754
17,Harvey Leigh’s Public House,Bar,NaN,AU,Highgate,Australia,NaN,6484,"[Highgate WA 6003, Australia]","[{u'lat': -31.938995, u'lng': 115.869441, u'label': u'display'}]",-31.938995,115.869441,NaN,6003,WA,5bb0ad293149b9002c0b4835


#### Great. Now let's explore the 1st pub: Durty Nelly's Irish Pub and see its reviews (tips in Foursquare)

In [75]:
venue_id = '4b383f8cf964a520274d25e3' # ID of Meena's fine foods
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# Send the GET request
result = requests.get(url).json()
result['response']['venue']

# Let's print the restaurant's ratings
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.1


In [76]:
## Location's Reviews
limit = 5 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
tips = results['response']['tips']['items']
tip = results['response']['tips']['items'][0]

# Formatting it neatly in a dataframe 
pd.set_option('display.max_colwidth', -1)
tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"Thursday is ""Spicy Beef Nachos"" day. Good value at $15. You'll be singing ""Nacho Nacho Man"" like Homer Simpson after plate of these!",0,0,55d46fa7498e8c25b44f42cb,Dylan,McArdle,male,3994715


### You don't get much results back from Foursquare using a free developer account.

## Conclusion

To wrap up this final assignment, let me summarise what we've done so far:

1) Create a dataset on our own (#10 suburbs in Perth) 

2) Load the dataset with latitudes and longitudes using Geocoder

3) Visualize the suburbs on a Folium map (We identified Baldivis as an outlier from the rest)

4) Get the top venues in each suburb into a Pandas dataframe so we can do a comparison

5) Regroup the dataframe to show each suburb's top 8 most common venues

6) From #5, we are able to identified a suburb and a kind of business to start up in

7) Floreat is an ideal place (near to city centre and airport), and not much competition for pubs!

8) Get all pubs in Floreat into a dataframe (Only 8 pubs available within a 5000m radius of the suburb!)

9) Get a review of the first pub




It's been a great enjoyment going through this course. 

Thank you for reading. :)